In [1]:
%matplotlib inline
from __future__ import division

import pandas as pd
import utils

In [2]:
visitors, events, devices, url_categories = utils.load_data()

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5,6,7,8,11,12,13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
event_categories = utils.categorize_events(events, url_categories)

In [4]:
visitors.identity_type.value_counts()

POI    4006149
AUT    1979658
Name: identity_type, dtype: int64

In [5]:
categorized_visits = pd.merge(visitors, event_categories, on='dg_id')

In [6]:
categorized_visits['authorized'] = categorized_visits.identity_type.apply(lambda x: 1 if x=='AUT' else 0)

In [7]:
pharma = categorized_visits[categorized_visits.site_category=='Pharma']

In [8]:
pharma_activity = pharma.groupby('url_y').authorized.agg(['sum', 'count']).reset_index()
pharma_activity.columns = ['url', 'authorized_visits', 'total_visits']

In [9]:
specialty_activity = categorized_visits.groupby('primary_specialty').authorized.agg(['sum', 'count']).reset_index()

In [10]:
specialty_activity

,primary_specialty,sum,count
0,00 General Practitioner,1759,1759
1,10 Oral Surgeon,12,12
2,15 Endodontist,20,20
3,20 Orthodontist,2,2
4,40 Periodontist,4,4
5,Abdominal Radiology (AR),22,22
6,Abdominal Surgery (AS),1,1
7,Addiction Medicine,369,369
8,Addiction Medicine (ADM),351,351
9,Addiction Psychiatry (ADP),397,397


In [11]:
pharma_activity = pd.merge(url_categories, pharma_activity, on='url')

In [12]:
pharma_activity['pct_authorized'] = pharma_activity.apply(lambda x: x.authorized_visits/x.total_visits, axis=1)

In [16]:
pharma_activity = pharma_activity.fillna('')

In [14]:
def authorized_visits(pharma_site, pharma_activity):
    f = pharma_activity[pharma_activity.url==pharma_site]
    print 'Pharma Site :: ' + pharma_site + ' :: ' + str(round(f.pct_authorized*100,2)) + '% authorized'
    disease_category = f.disease_category.item()
    fd = pharma_activity[pharma_activity.disease_category==disease_category]
    if len(fd) >= 3:
        authorized = sum(fd.authorized_visits)
        total = sum(fd.total_visits)
        print 'Disease Category :: ' + disease_category + ' :: ' + str(round(authorized/total*100, 2)) + '% authorized'
        diseases = [x.lower().strip() for x in f.disease.item().split(',') if x.strip() != '']
        for d in diseases:
            authorized = 0
            total = 0
            count = 0
            for row in fd.iterrows():
                r = row[1]
                if d in r.disease.lower():
                    authorized += r.authorized_visits
                    total += r.total_visits
                    count += 1
            if count >= 3:
                print 'Disease :: ' + d + ' :: ' + str(round(authorized/total*100, 2)) + '% authorized'
                

In [17]:
for url in pharma_activity.url.sample(10):
    authorized_visits(url, pharma_activity)

Pharma Site :: www.tyvaso.com :: 71.43% authorized
Multiple :: 75.91% authorized
Pharma Site :: www.biooncology.com :: 46.74% authorized
Disease Category :: Oncology :: 67.82% authorized
Disease :: cancer :: 67.77% authorized
Pharma Site :: https://www.merckconnect.com/noxafil/ :: 57.84% authorized
Disease Category :: Oncology  :: 68.66% authorized
Pharma Site :: https://www.merckaccessprogram-zepatier.com/hcp/ :: 82.81% authorized
Disease Category :: Infectious Disease :: 73.44% authorized
Pharma Site :: http://www.pomalyst.com/hcp/ :: 84.42% authorized
Disease Category :: Oncology  :: 68.66% authorized
Pharma Site :: www.trendsinoncology.com :: 100.0% authorized
Disease Category :: Oncology :: 67.82% authorized
Pharma Site :: www.stopstalling.com :: 65.38% authorized
Disease Category :: Urology :: 74.94% authorized
Disease :: over active bladder :: 74.94% authorized
Pharma Site :: www.alecensa.com :: 54.84% authorized
Disease Category :: Oncology :: 67.82% authorized
Pharma Site :: w

In [22]:
for url in pharma_activity[pharma_activity.disease_category=='Oncology'].url.sample(3):
    authorized_visits(url, pharma_activity)

Pharma Site :: www.avastin.com :: 72.65% authorized
Disease Category :: Oncology :: 67.82% authorized
Disease :: non_small cell lung cancer (nsclc) :: 67.2% authorized
Pharma Site :: https://www.keytruda.com/hcp/ :: 69.92% authorized
Disease Category :: Oncology :: 67.82% authorized
Disease :: melanoma :: 67.09% authorized
Disease :: non_small cell lung cancer (nsclc) :: 67.2% authorized
Pharma Site :: www.knowyourprostateplan.com :: 57.95% authorized
Disease Category :: Oncology :: 67.82% authorized
Disease :: prostate cancer :: 68.13% authorized


In [19]:
pharma_activity

,url,site_category,site_sub_category,disease_category,disease,pharma_firm,stripped_url,clean_url,authorized_visits,total_visits,pct_authorized
0,http://www.adempas-us.com,Pharma,Branded Consumer,"Cardiology, Pulmonology\rMultiple","Thromboembolic Pulmonary Hypertension, Pulmona...",Bayer,adempas-us,adempas-us.com,48,63,0.761905
1,http://www.adempas-us.com/hcp/,Pharma,Branded HCP,"Cardiology, Pulmonology\rMultiple","Thromboembolic Pulmonary Hypertension, Pulmona...",Bayer,adempas-us/hcp,adempas-us.com/hcp,50,62,0.806452
2,http://www.herceptin.com/hcp/,Pharma,Branded HCP,Oncology,"Metastatic Breast Cancer, Metastatic Gastric C...",Genentech,herceptin/hcp,herceptin.com/hcp,79,133,0.593985
3,http://www.journeyforcontrol.com/,Pharma,Unbranded HCP,Endocrinology,Diabetes,Merck,journeyforcontrol,journeyforcontrol.com,903,1274,0.708791
4,http://www.kadcyla.com/hcp/,Pharma,Branded HCP,Oncology,metastatic breast cancer,Genentech,kadcyla/hcp,kadcyla.com/hcp,82,112,0.732143
5,http://www.lenvima.com/hcp/,Pharma,Branded HCP,Oncology,"Thyroid Cancer, advanced renal cell carcinoma ...",Eisai,lenvima/hcp,lenvima.com/hcp,297,334,0.889222
6,http://www.perjeta.com/hcp/,Pharma,Branded HCP,Oncology,metastatic breast cancer,Genentech,perjeta/hcp,perjeta.com/hcp,469,646,0.726006
7,http://www.pomalyst.com/hcp/,Pharma,Branded HCP,Oncology,multiple myeloma,Celgene,pomalyst/hcp,pomalyst.com/hcp,168,199,0.844221
8,http://www.rituxan.com/hem/hcp,Pharma,Branded HCP,"Oncology, Rheumatology\rMultiple","non-Hodgkin's lymphoma, �chronic lymphocytic l...",Genentech,rituxan/hcp,rituxan.com/hem/hcp,124,170,0.729412
9,http://www.tarceva.com/hcp/,Pharma,Branded HCP,Oncology,"non-small cell lung cancer (NSCLC), advanced-s...",Genentech,tarceva/hcp,tarceva.com/hcp,59,88,0.670455


In [23]:
visitors['has_']

Index([u'Unnamed: 0', u'dg_id', u'identity_type', u'professional_designation',
       u'npi_number', u'primary_specialty', u'primary_specialty_group',
       u'secondary_specialty', u'secondary_specialty_group', u'first_name',
       u'last_name', u'formatted_name', u'city', u'state', u'zipcode',
       u'birth_year', u'grad_year', u'grad_school', u'gender',
       u'suspected_bot', u'exclude'],
      dtype='object')

In [24]:
pharma_activity

,url,site_category,site_sub_category,disease_category,disease,pharma_firm,stripped_url,clean_url
0,http://www.paradigmspine.com/content/are-you-a...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine,paradigmspine.com/content/are-you-a-candidate
1,http://www.paradigmspine.com/content/back-spin...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine,paradigmspine.com/content/back-spine-overview
2,http://www.paradigmspine.com/content/coflex-study,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine,paradigmspine.com/content/coflex-study
3,http://www.paradigmspine.com/content/degenerat...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine,paradigmspine.com/content/degenerative-disc-di...
4,http://www.paradigmspine.com/content/degenerat...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine,paradigmspine.com/content/degenerative-disc-di...
5,http://www.paradigmspine.com/content/glossary,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine,paradigmspine.com/content/glossary
6,http://www.paradigmspine.com/content/patients-...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine,paradigmspine.com/content/patients-like-you-fd...
7,http://www.paradigmspine.com/content/patient-t...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine,paradigmspine.com/content/patient-testimonials
8,http://www.paradigmspine.com/content/spinal-st...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine,paradigmspine.com/content/spinal-stenosis-surg...
9,http://www.paradigmspine.com/content/spondylol...,Med Device,Branded Consumer,Surgery,NaN,Paradigm Spine,paradigmspine,paradigmspine.com/content/spondylolisthesis
